In [5]:
import os, shutil
import pdal, gdal
import geopandas as gpd
import numpy as np
import rasterio as rio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling
from datetime import datetime
from pathlib import Path
from joblib import Parallel, delayed
from scipy import ndimage as ndi

## Functions

In [213]:
def developOverviews(file):
    os.system(f"gdalinfo -mm {file}")
    os.system(f"gdaladdo -clean {file}")
    os.system(f"gdaladdo -r average -ro --config COMPRESS_OVERVIEW LZW -minsize 8 {file}")

    
def calculateHAG(dsm, out_dir, overwrite=False):
    ofile = os.path.join(out_dir, os.path.basename(dsm).replace(".tif","_HAG.tif"))
        
    if os.path.exists(ofile) and overwrite==False:
        return ofile
    
    with rio.open(dsm) as src:
        bnds = src.bounds
        swidth = src.width
        sheight = src.height
        kwargs= src.profile
        sres  = src.res[0]
        trans = src.transform
        
    #xoff = int((tx - bnds.left)/sres)
    #yoff = int((bnds.top - ty)/sres) 
    #print("xoff, yoff", xoff, yoff)
    
    with rio.open(loc_dsmVRT) as src:
        dsm_ft = src.read(1, window=from_bounds(bnds.left, bnds.bottom, bnds.right, bnds.top, transform=src.transform, height=sheight, width=swidth).round_offsets(),
                          out_shape=(swidth,sheight))
        #print("DSM Sample Point:", [i[0] for i in src.sample([(tx,ty)])])

    with rio.open(loc_demVRT) as src:
        demBnds = src.bounds
        demRes = src.res[0]
        
        offsetL = demRes - (demBnds.left - bnds.left) % demRes
        offsetR = (demBnds.right - bnds.right) % demRes
        offsetT = (demBnds.top - bnds.top) % demRes
        offsetB = demRes - (demBnds.bottom - bnds.bottom) % demRes
                
        read_window = from_bounds(bnds.left-offsetL, bnds.bottom-offsetB, bnds.right+offsetR, bnds.top+offsetT, transform=src.transform)
        
        otransform = src.window_transform(read_window)
        buffW = swidth + round(offsetL/sres) + round(offsetR/sres)
        buffH = sheight + round(offsetB/sres) + round(offsetT/sres)
        
        dem_m = src.read(1, window=read_window,
                         out_shape=(buffW,buffH))
        
        #trim out buffer now that data has been resampled at read in 
        dem_m = dem_m[round(offsetL/sres):-round(offsetR/sres),
                      round(offsetT/sres):-round(offsetB/sres)]
        
        
        nd = src.nodata
        dem_m[dem_m==nd] = np.NaN
        dem_ft = dem_m/0.3048
        #print("DEM Sample Point:", [i[0]/0.3048 for i in src.sample([(tx,ty)])])
        
    #xoff = -3
    #yoff = 3
    #print("DSM ARRAY", dsm_ft[yoff,xoff])
    #print("DEM ARRAY", dem_ft[yoff,xoff])
    
    hag_a = dsm_ft - dem_ft
    #print("HAG VALUE: ", hag_a[yoff, xoff])
    # convert to absolute distance
    hag_a = np.abs(hag_a).astype(np.uint16)
    # force to zero (no negatives)
    #hag_a = hag_a.clip(min=0).astype(np.uint16)

    kwargs.update(dtype=hag_a.dtype, nodata=np.iinfo(np.uint16).max)
    #kwargs.update(dtype=hag_a.dtype, nodata=-9999)

    hag_a[np.isnan(dem_m)] = kwargs['nodata']
    with rio.open(ofile,"w", **kwargs) as dst:
        dst.write(hag_a,1)

    developOverviews(ofile)

    print(f"Created HAG file: {ofile}")
    #with rio.open(ofile) as src:
    #    print("HAG Sample Point:", [i[0] for i in src.sample([(tx,ty)])])
    
    return ofile

    #except:
    #    return f"Failed for dsm {dsm}"


def calculateDTG(dsm, hag_dir, dtg_dir, threshold=4, overwrite=False):
    hag_file = calculateHAG(dsm, hag_dir, overwrite=overwrite)
    
    ofile = os.path.join(dtg_dir, os.path.basename(hag_file).replace(".tif","_distance.tif"))
    
    if os.path.exists(ofile) and overwrite==False:
        return ofile
    
    with rio.open(hag_file) as src:
        kwargs = src.profile
        hag_a = src.read(1)
        res = src.res[0]
    hag_a = np.where(hag_a>=threshold,1,0)
    
    #calculate euclidean distance to ground
    distance = ndi.distance_transform_edt(hag_a)
    #convert to feet
    distance = distance * res
    #take ceiling of float
    distance = np.ceil(distance)
    
    kwargs.update(dtype=np.int32)
    with rio.open(ofile, 'w', **kwargs) as dst:
        dst.write(distance.astype(np.int32),1)
        
    developOverviews(ofile)
    
    return ofile

In [35]:
def testSlope(dsm):
    
    with rio.open(dsm) as src:
        bnds = src.bounds
        print(bnds)
        swidth = src.width
        sheight = src.height
        kwargs= src.profile
        sres  = src.res[0]
        print(sres)
        trans = src.transform
    
    xoff = int((tx - bnds.left)/sres)
    yoff = int((bnds.top - ty)/sres) 
    print("xoff, yoff", xoff, yoff)
    
    with rio.open(loc_slope) as src:
        slopeBnds = src.bounds
        slopeRes = src.res[0]
        
        offsetL = slopeRes - (slopeBnds.left - bnds.left) % slopeRes
        offsetR = (slopeBnds.right - bnds.right) % slopeRes
        offsetT = (slopeBnds.top - bnds.top) % slopeRes
        offsetB = slopeRes - (slopeBnds.bottom - bnds.bottom) % slopeRes
                
        read_window = from_bounds(bnds.left-offsetL, bnds.bottom-offsetB, bnds.right+offsetR, bnds.top+offsetT, transform=src.transform)
        
        buffW = swidth + round(offsetL/sres) + round(offsetR/sres)
        buffH = sheight + round(offsetB/sres) + round(offsetT/sres)
        
        slope = src.read(1, window=read_window,
                         out_shape=(buffW,buffH))
        
        #trim out buffer now that data has been resampled at read in 
        slope = slope[round(offsetL/sres):-round(offsetR/sres),
                      round(offsetT/sres):-round(offsetB/sres)]
        
        print("Slope Sample Point:", [i[0] for i in src.sample([(tx,ty)])])
        print("DSM ARRAY CLIPPED", slope[yoff,xoff])
        
        read_window = from_bounds(bnds.left, bnds.bottom, bnds.right, bnds.top, transform=src.transform)
        
        slope = src.read(1, window=read_window,
                         out_shape=(buffW,buffH))
        print("DSM ARRAY UNCLIPPED", slope[yoff,xoff])
        
    print(slope.shape)
    #xoff = -3
    #yoff = 3
    print("DSM ARRAY", slope[yoff,xoff])
    
    
    
    return

loc_dsm = '../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/'
loc_dsmVRT = os.path.abspath("../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/EPC_DSM80cm_2019.vrt")
dsms = [os.path.join(loc_dsm,f) for f in os.listdir(loc_dsm) if f.endswith(".tif")]
loc_slope = "../OtherData/10mDEMs/DEM10mNED_slope.tif"

tx, ty = 1078688.16,434277.53
for file in dsms:
    if "E1080_N430" not in file:
        continue
    else:
        testSlope(file)

BoundingBox(left=1059739.62, bottom=429700.22000000003, right=1080261.91473, top=450222.51473000005)
2.62467
xoff, yoff 7219 6075
Slope Sample Point: [4.2218804]
DSM ARRAY CLIPPED 4.2218804
DSM ARRAY UNCLIPPED 4.643389
(7822, 7834)
DSM ARRAY 4.643389


In [26]:
(tx-779739.61)/2.62

113126.22519083972

In [13]:
loc_dsm = '../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/'
dsms = [os.path.join(loc_dsm,f) for f in os.listdir(loc_dsm) if f.endswith(".tif")]

loc_demVRT = os.path.abspath("../OtherData/10mDEMs/DEM10mNED.vrt")
loc_dsmVRT = os.path.abspath("../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/EPC_DSM80cm_2019.vrt")
loc_hag = os.path.abspath("../EPCExtent_30cm/Elevation_80cmNPS/HAG_NED_80cm")
loc_dtg = os.path.abspath("../EPCExtent_30cm/Elevation_80cmNPS/HAG_NED_80cm_DTG")

os.makedirs(loc_hag, exist_ok=True)
os.makedirs(loc_dtg, exist_ok=True)


#hags = Parallel(n_jobs=10, verbose=5)(delayed(calculateHAG)(file, loc_hag, overwrite=False) for file in dsms)
#print("Finished with HAGS")
distance_files = Parallel(n_jobs=10, verbose=10)(delayed(calculateDTG)(file, loc_hag, loc_dtg, threshold=4, overwrite=True) for file in dsms)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


NameError: name 'calculateDTG' is not defined

In [216]:
vrt = gdal.BuildVRT(f"{loc_dtg}/EPC_HAGNEDDTG80cm_2019.vrt", distance_files)
vrt = None
print("FINISHED")

FINISHED


-------------------------